In [1]:
%%time
import pandas as pd
#from sklearn import cross_validation
#from sklearn.feature_selection import SelectPercentile, f_classif
#from sklearn.linear_model import Lasso
#from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.naive_bayes import GaussianNB
from sklearn import tree
#from sklearn.svm import SVC
#from sklearn import ensemble
#from sklearn.naive_bayes import BernoulliNB
#from datetime import datetime
#from blaze import Data
#import dask.dataframe as dd
#import dask.array as da

CPU times: user 688 ms, sys: 135 ms, total: 823 ms
Wall time: 872 ms


In [2]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

CPU times: user 12 µs, sys: 4 µs, total: 16 µs
Wall time: 17.2 µs


In [3]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

CPU times: user 1min 57s, sys: 1min 35s, total: 3min 33s
Wall time: 3min 43s


In [4]:
%%time
all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(all_train['orig_destination_distance'].median()).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
all_train = all_train.drop(['date_time','srch_ci','srch_co','cnt'], 1)
all_train = all_train.reindex_axis(sorted(all_train.columns), axis=1)

CPU times: user 11.3 s, sys: 37.3 s, total: 48.6 s
Wall time: 59.9 s


In [5]:
all_train.head()

,channel,hotel_cluster,hotel_continent,hotel_country,hotel_market,is_booking,is_mobile,is_package,orig_destination_distance,posa_continent,site_name,srch_adults_cnt,srch_children_cnt,srch_destination_id,srch_destination_type_id,srch_rm_cnt,user_id,user_location_city,user_location_country,user_location_region
0,9,1,2,50,628,0,0,1,2234,3,2,2,0,8250,1,1,12,48862,66,348
1,9,1,2,50,628,1,0,1,2234,3,2,2,0,8250,1,1,12,48862,66,348
2,9,1,2,50,628,0,0,0,2234,3,2,2,0,8250,1,1,12,48862,66,348
3,3,80,2,50,1457,0,0,0,913,3,2,2,0,14984,1,1,93,35390,66,442
4,3,21,2,50,1457,0,0,0,913,3,2,2,0,14984,1,1,93,35390,66,442


In [8]:
%%time
#Split into training and test data
#features = all_train.ix[:,:'hotel_market']
features = all_train.drop(['hotel_cluster'], 1) 
labels = all_train['hotel_cluster']

CPU times: user 2.04 s, sys: 7.23 s, total: 9.26 s
Wall time: 12.1 s


In [ ]:
%%time
#features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.25, random_state=42)

In [9]:
%%time
selector = SelectKBest(k=12)
#.fit_transform(X, y)
#X_new.shape

selector.fit(features, labels.values.ravel())

CPU times: user 1min 3s, sys: 2min 52s, total: 3min 56s
Wall time: 5min 37s


In [16]:
%%time
feature_list = selector.transform(features)

CPU times: user 1.53 s, sys: 3.43 s, total: 4.96 s
Wall time: 6.35 s


In [12]:
feature_list[:5]

array([[    2,    50,   628,     0,     1,  2234,     3,     2,     0,
         8250,     1,   348],
       [    2,    50,   628,     1,     1,  2234,     3,     2,     0,
         8250,     1,   348],
       [    2,    50,   628,     0,     0,  2234,     3,     2,     0,
         8250,     1,   348],
       [    2,    50,  1457,     0,     0,   913,     3,     2,     0,
        14984,     1,   442],
       [    2,    50,  1457,     0,     0,   913,     3,     2,     0,
        14984,     1,   442]])

In [14]:
features.head()

,channel,hotel_continent,hotel_country,hotel_market,is_booking,is_mobile,is_package,orig_destination_distance,posa_continent,site_name,srch_adults_cnt,srch_children_cnt,srch_destination_id,srch_destination_type_id,srch_rm_cnt,user_id,user_location_city,user_location_country,user_location_region
0,9,2,50,628,0,0,1,2234,3,2,2,0,8250,1,1,12,48862,66,348
1,9,2,50,628,1,0,1,2234,3,2,2,0,8250,1,1,12,48862,66,348
2,9,2,50,628,0,0,0,2234,3,2,2,0,8250,1,1,12,48862,66,348
3,3,2,50,1457,0,0,0,913,3,2,2,0,14984,1,1,93,35390,66,442
4,3,2,50,1457,0,0,0,913,3,2,2,0,14984,1,1,93,35390,66,442


In [ ]:
%%time
clf = tree.DecisionTreeClassifier(min_samples_split=250)
#clf = GaussianNB()
#clf = ensemble.AdaBoostClassifier(n_estimators=10)
#clf = ensemble.AdaBoostClassifier(SVC(probability=True, kernel='linear'),n_estimators=10)
#clf = ensemble.GradientBoostingClassifier(SVC(probability=True, kernel='linear'),n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=3)
#clf = ensemble.RandomForestClassifier(n_estimators=5)
#clf - BernoulliNB()
clf = clf.fit(feature_list, labels.values.ravel())

In [ ]:
%%time
#feature_test_list = selector.transform(features_test)
#pred = clf.predict(feature_test_list)

In [ ]:
#print accuracy_score(pred, labels_test.values.ravel())

In [21]:
%%time
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

CPU times: user 12 µs, sys: 5 µs, total: 17 µs
Wall time: 16.2 µs


In [22]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

CPU times: user 7.07 s, sys: 1.21 s, total: 8.28 s
Wall time: 8.41 s


In [23]:
all_test.head()

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


In [24]:
%%time
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(all_test['orig_destination_distance'].median()).astype(int)
all_test['is_booking'] = 1
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
all_test = all_test.drop(['id', 'date_time','srch_ci','srch_co'], 1)
all_test = all_test.reindex_axis(sorted(all_test.columns), axis=1)
#testing_file = testing_file.values

CPU times: user 644 ms, sys: 725 ms, total: 1.37 s
Wall time: 1.56 s


In [25]:
all_test.head()

,channel,hotel_continent,hotel_country,hotel_market,is_booking,is_mobile,is_package,orig_destination_distance,posa_continent,site_name,srch_adults_cnt,srch_children_cnt,srch_destination_id,srch_destination_type_id,srch_rm_cnt,user_id,user_location_city,user_location_country,user_location_region
0,3,6,204,27,1,1,0,5539,3,2,2,0,12243,6,1,1,37449,66,174
1,10,6,204,1540,1,1,0,5873,3,2,2,0,14474,7,1,1,37449,66,174
2,1,2,50,699,1,0,0,3975,3,2,4,0,11353,1,1,20,17440,66,142
3,10,2,50,628,1,0,1,1508,3,2,2,0,8250,1,1,28,34156,66,258
4,0,2,50,538,1,0,0,66,3,2,2,0,11812,1,1,50,36345,66,467


In [ ]:
%%time
feature_test_file = selector.transform(all_test)
#pred = clf.predict(feature_test_list)
probs = pd.DataFrame(clf.predict_proba(feature_test_file))

In [ ]:
%%time
probs_series = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in probs.iterrows()])
probs_series = probs_series.values

In [ ]:
%%time
indices = [a for a,b in probs_series]
values = [b for a,b in probs_series]

In [ ]:
%%time
submission = pd.DataFrame()
submission['id'] = indices
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values]

In [ ]:
submission.head()

In [ ]:
%%time
submission.to_csv('submission.csv', index=False)